In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels # prepare for next layer
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers) # pass layers to nn.Sequential

In [5]:
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(*conv_blks, nn.Flatten(),
                        nn.Linear(out_channels*7*7, 4096), nn.ReLU(), nn.Dropout(0.5),
                        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
                        nn.Linear(4096, 10))

In [11]:
# vgg architecture--double the #channel while the the size of image decreasing
# VGG-11
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

net = vgg(conv_arch)

In [12]:
X = torch.rand((1,1,224,224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'Output Shape: \t', X.shape)

Sequential Output Shape: 	 torch.Size([1, 64, 112, 112])
Sequential Output Shape: 	 torch.Size([1, 128, 56, 56])
Sequential Output Shape: 	 torch.Size([1, 256, 28, 28])
Sequential Output Shape: 	 torch.Size([1, 512, 14, 14])
Sequential Output Shape: 	 torch.Size([1, 512, 7, 7])
Flatten Output Shape: 	 torch.Size([1, 25088])
Linear Output Shape: 	 torch.Size([1, 4096])
ReLU Output Shape: 	 torch.Size([1, 4096])
Dropout Output Shape: 	 torch.Size([1, 4096])
Linear Output Shape: 	 torch.Size([1, 4096])
ReLU Output Shape: 	 torch.Size([1, 4096])
Dropout Output Shape: 	 torch.Size([1, 4096])
Linear Output Shape: 	 torch.Size([1, 10])


In [14]:
# VGG-11 is too big, so we just construct a smaller net with fewer #channel
ratio = 4
small_conv_arch = [(pair[0], pair[1] // 4) for pair in conv_arch]
net = vgg(small_conv_arch)

In [15]:
X = torch.rand((1,1,224,224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, 'Output Shape: \t', X.shape)

Sequential Output Shape: 	 torch.Size([1, 16, 112, 112])
Sequential Output Shape: 	 torch.Size([1, 32, 56, 56])
Sequential Output Shape: 	 torch.Size([1, 64, 28, 28])
Sequential Output Shape: 	 torch.Size([1, 128, 14, 14])
Sequential Output Shape: 	 torch.Size([1, 128, 7, 7])
Flatten Output Shape: 	 torch.Size([1, 6272])
Linear Output Shape: 	 torch.Size([1, 4096])
ReLU Output Shape: 	 torch.Size([1, 4096])
Dropout Output Shape: 	 torch.Size([1, 4096])
Linear Output Shape: 	 torch.Size([1, 4096])
ReLU Output Shape: 	 torch.Size([1, 4096])
Dropout Output Shape: 	 torch.Size([1, 4096])
Linear Output Shape: 	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.05, 8, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

training on cpu
